In [2]:
import numpy as np

class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)


In [3]:
from data_reader import read_interaction_matrix
R = read_interaction_matrix()

In [22]:
m = MF(R, 64, 0.1, 0.01, 2000)
m.train()

Iteration: 10 ; error = 0.1481
Iteration: 20 ; error = 0.0876
Iteration: 30 ; error = 0.0560
Iteration: 40 ; error = 0.0378
Iteration: 50 ; error = 0.0264
Iteration: 60 ; error = 0.0190
Iteration: 70 ; error = 0.0140
Iteration: 80 ; error = 0.0105
Iteration: 90 ; error = 0.0080
Iteration: 100 ; error = 0.0061
Iteration: 110 ; error = 0.0048
Iteration: 120 ; error = 0.0037
Iteration: 130 ; error = 0.0030
Iteration: 140 ; error = 0.0024
Iteration: 150 ; error = 0.0019
Iteration: 160 ; error = 0.0015
Iteration: 170 ; error = 0.0012
Iteration: 180 ; error = 0.0010
Iteration: 190 ; error = 0.0008
Iteration: 200 ; error = 0.0007
Iteration: 210 ; error = 0.0005
Iteration: 220 ; error = 0.0004
Iteration: 230 ; error = 0.0004
Iteration: 240 ; error = 0.0003
Iteration: 250 ; error = 0.0002
Iteration: 260 ; error = 0.0002
Iteration: 270 ; error = 0.0002
Iteration: 280 ; error = 0.0001
Iteration: 290 ; error = 0.0001
Iteration: 300 ; error = 0.0001
Iteration: 310 ; error = 0.0001
Iteration: 320 ; 

[(0, 0.2670461815937577),
 (1, 0.24717799916266162),
 (2, 0.2304337244952816),
 (3, 0.21494385061556623),
 (4, 0.20101170050059264),
 (5, 0.18840154585764196),
 (6, 0.17693049463976324),
 (7, 0.16635239057406503),
 (8, 0.15672134828505896),
 (9, 0.14806669129417102),
 (10, 0.139809489871958),
 (11, 0.1321674808056096),
 (12, 0.12508344130103952),
 (13, 0.11851542606717708),
 (14, 0.11234761198073719),
 (15, 0.10676134342007862),
 (16, 0.10153754120056155),
 (17, 0.09650079175083744),
 (18, 0.09188097808765476),
 (19, 0.08760654825691598),
 (20, 0.0835650151105769),
 (21, 0.07972720600900311),
 (22, 0.07608611040061013),
 (23, 0.07262549595313784),
 (24, 0.06940154831871863),
 (25, 0.0664827096854909),
 (26, 0.06363096352466985),
 (27, 0.061008459769406484),
 (28, 0.05842765693665891),
 (29, 0.055978765949139565),
 (30, 0.05375209763230915),
 (31, 0.051573623492417585),
 (32, 0.04954193884124659),
 (33, 0.04761447653283221),
 (34, 0.04576358126180168),
 (35, 0.04398344276090232),
 (36, 

In [18]:
T = m.full_matrix()

In [21]:
R.sum(), T.sum()

(23038.0, 982226.8322417325)